In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt

In [2]:
tabela = pd.read_csv("esquistossomose.csv")
tabela

,TP_NOT,ID_AGRAVO,DT_NOTIFIC,SEM_NOT,NU_ANO,SG_UF_NOT,ID_MUNICIP,ID_REGIONA,ID_UNIDADE,DT_SIN_PRI,...,DT_TRANSSM,DT_TRANSRS,DT_TRANSSE,DT_OBITO,DS_FORMA,AN_QUALI,DTTRAT,DT_RESU3,OUTRO_EX,IDADE
0,2,B659,20180101,201801,2018,31,310620,1449,22659,20171229,...,20180607,,,,,,20180115,20180425,KATO-KATZ POSITIVO,55.0
1,2,B659,20180102,201801,2018,31,315210,1458,2218038,20171201,...,20180111,,,,,1,20180102,,,30.0
2,2,B659,20180102,201801,2018,31,312270,1458,6300219,20171207,...,20180215,,,,,1,20180102,,,33.0
3,2,B659,20180102,201801,2018,26,260830,1500,6511465,20171226,...,,,,,,1,20180108,,,52.0
4,2,B659,20180102,201801,2018,32,320130,1510,2546892,20171205,...,,,,,,1,20180102,,,46.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4214,2,B659,20181228,201852,2018,31,310620,1449,23183,20181005,...,20190221,,,,NAO INFORMADA,3,20181228,,BIOPSIA RETAL PESQUISA DE OVOS DO SCHIS,39.0
4215,2,B659,20181228,201852,2018,26,261160,1497,655,20181120,...,,,,,,,,,,16.0
4216,2,B659,20181229,201852,2018,31,313130,1468,2205440,20181001,...,,,,,,2,20181030,,IMUNOFLUORESCENCIA,52.0
4217,2,B659,20181230,201901,2018,32,320435,1509,2494353,20181226,...,20190624,,,,,1,20190102,20190522,PARASITOLOGICO,59.0


In [3]:
#Quantos registros existem?
len(tabela.index)

4219

In [4]:
def questao_1(datapath):
    tabela = pd.read_csv(datapath)
    tamanho = len(tabela.index)
    return tamanho

In [5]:
questao_1("esquistossomose.csv")

4219

# Questão 3

In [18]:
casos_por_sexo = dict(tabela["CS_SEXO"].value_counts())
maior_quantidade = list(casos_por_sexo.keys())
tupla = tuple(maior_quantidade[0])
print(casos_por_sexo)
print(tupla)

{'M': 2558, 'F': 1661}
('M',)


# Questão 7

In [23]:
mun_por_est = {31 : 853, 35 : 645, 43 : 497, 29 : 417, 41 : 399, 42 : 295, 52 : 246, 22 : 224, 25 : 223, 21: 217, 26 : 184, 23 : 184, 24 : 167, 15 : 144, 51 : 141,
17 : 139, 27 : 102, 33 : 92, 50 : 79, 32 : 78, 28 : 75, 13 : 62,11 : 52, 12 : 22, 16 : 16, 14 : 15}
codigo_estados = {35: ' SP', 41: ' PR', 42: ' SC', 43: ' RS', 50: ' MS', 11: ' RO', 12: ' AC', 13: ' AM',14: ' RR', 15: ' PA', 16: ' AP', 17: ' TO', 21: ' MA', 24: ' RN', 25: ' PB', 26: ' PE', 27: ' AL', 28: ' SE', 29: ' BA', 31: ' MG', 33: ' RJ', 51: ' MT', 52: ' GO', 53: ' DF', 22: ' PI', 23: ' CE', 32: ' ES'}

casos_por_estado = tabela.groupby('SG_UF_NOT')['ID_MUNICIP'].nunique()
tabela_auxiliar = pd.DataFrame({"COD": mun_por_est.keys(), "Quantidade de Cidades": mun_por_est.values()})
tabela_auxiliar["Quantidade de casos"] = tabela_auxiliar["COD"].map(casos_por_estado).fillna(0)
tabela_auxiliar["Porcentagem"] = (tabela_auxiliar["Quantidade de casos"]*100/tabela_auxiliar["Quantidade de Cidades"]).round(2)
tabela_auxiliar["UF"] = tabela_auxiliar["COD"].map(codigo_estados)
dicionario = dict(zip(tabela_auxiliar["UF"], tabela_auxiliar["Porcentagem"]))
dicionario


,COD,Quantidade de Cidades
0,31,853
1,35,645
2,43,497
3,29,417
4,41,399
5,42,295
6,52,246
7,22,224
8,25,223
9,21,217


In [13]:
def questao_7(datapath):
    tabela = pd.read_parquet(datapath)
    mun_por_est = {31 : 853, 35 : 645, 43 : 497, 29 : 417, 41 : 399, 42 : 295, 52 : 246, 22 : 224, 25 : 223, 21: 217, 26 : 184, 23 : 184, 24 : 167, 15 : 144, 51 : 141, 17 : 139, 27 : 102, 33 : 92, 50 : 79, 32 : 78, 28 : 75, 13 : 62,11 : 52, 12 : 22, 16 : 16, 14 : 15}
    codigo_estados = {35: ' SP', 41: ' PR', 42: ' SC', 43: ' RS', 50: ' MS', 11: ' RO', 12: ' AC', 13: ' AM',14: ' RR', 15: ' PA', 16: ' AP', 17: ' TO', 21: ' MA', 24: ' RN', 25: ' PB', 26: ' PE', 27: ' AL', 28: ' SE', 29: ' BA', 31: ' MG', 33: ' RJ', 51: ' MT', 52: ' GO', 53: ' DF', 22: ' PI', 23: ' CE', 32: ' ES'}

    casos_por_estado = tabela.groupby('SG_UF_NOT')['ID_MUNICIP'].nunique()
    tabela_auxiliar = pd.DataFrame({"COD": mun_por_est.keys(), "Quantidade de Cidades": mun_por_est.values()})
    tabela_auxiliar["Quantidade de casos"] = tabela_auxiliar["COD"].map(casos_por_estado).fillna(0)
    tabela_auxiliar["Porcentagem"] = (tabela_auxiliar["Quantidade de casos"]*100/tabela_auxiliar["Quantidade de Cidades"]).round(2)
    tabela_auxiliar["UF"] = tabela_auxiliar["COD"].map(codigo_estados)
    dicionario = dict(zip(tabela_auxiliar["UF"], tabela_auxiliar["Porcentagem"]))
    return dicionario

In [15]:
questao_7("esquistossomose.parquet")

{' MG': 0.0,
 ' SP': 0.0,
 ' RS': 0.0,
 ' BA': 0.0,
 ' PR': 0.0,
 ' SC': 0.0,
 ' GO': 0.0,
 ' PI': 0.0,
 ' PB': 0.0,
 ' MA': 0.0,
 ' PE': 0.0,
 ' CE': 0.0,
 ' RN': 0.0,
 ' PA': 0.0,
 ' MT': 0.0,
 ' TO': 0.0,
 ' AL': 0.0,
 ' RJ': 0.0,
 ' MS': 0.0,
 ' ES': 0.0,
 ' SE': 0.0,
 ' AM': 0.0,
 ' RO': 0.0,
 ' AC': 0.0,
 ' AP': 0.0,
 ' RR': 0.0}

In [17]:
tabela = pd.read_parquet("esquistossomose.parquet")
mun_por_est = {31 : 853, 35 : 645, 43 : 497, 29 : 417, 41 : 399, 42 : 295, 52 : 246, 22 : 224, 25 : 223, 21: 217, 26 : 184, 23 : 184, 24 : 167, 15 : 144, 51 : 141, 17 : 139, 27 : 102, 33 : 92, 50 : 79, 32 : 78, 28 : 75, 13 : 62,11 : 52, 12 : 22, 16 : 16, 14 : 15}
codigo_estados = {35: ' SP', 41: ' PR', 42: ' SC', 43: ' RS', 50: ' MS', 11: ' RO', 12: ' AC', 13: ' AM',14: ' RR', 15: ' PA', 16: ' AP', 17: ' TO', 21: ' MA', 24: ' RN', 25: ' PB', 26: ' PE', 27: ' AL', 28: ' SE', 29: ' BA', 31: ' MG', 33: ' RJ', 51: ' MT', 52: ' GO', 53: ' DF', 22: ' PI', 23: ' CE', 32: ' ES'}

casos_por_estado = tabela.groupby('SG_UF_NOT')['ID_MUNICIP'].nunique()
tabela_auxiliar = pd.DataFrame({"COD": mun_por_est.keys(), "Quantidade de Cidades": mun_por_est.values()})
tabela_auxiliar["Quantidade de casos"] = tabela_auxiliar["COD"].map(casos_por_estado).fillna(0)
tabela_auxiliar["Porcentagem"] = (tabela_auxiliar["Quantidade de casos"]*100/tabela_auxiliar["Quantidade de Cidades"]).round(2)
tabela_auxiliar["UF"] = tabela_auxiliar["COD"].map(codigo_estados)
dicionario = dict(zip(tabela_auxiliar["UF"], tabela_auxiliar["Porcentagem"]))

In [10]:
tabela = pd.read_parquet("esquistossomose.parquet")
tabela["DT_NOTIFICACAO"] = pd.to_datetime(tabela["DT_NOTIFIC"])
tabela["DT_SINTOMAS"] = pd.to_datetime(tabela["DT_SIN_PRI"])
tabela["ATRASO_NOT"] = tabela["DT_NOTIFICACAO"] - tabela["DT_SINTOMAS"]
# print(tabela["ATRASO_NOT"])
media_atraso = tabela.groupby("SG_UF_NOT")["ATRASO_NOT"].mean()
desvio_padrao = tabela.groupby("SG_UF_NOT")["ATRASO_NOT"].std()


#print(media_atraso)
print(desvio_padrao)
    

SG_UF_NOT
11   2277 days 17:05:58.792626080
14      7 days 19:26:45.571998334
15     18 days 10:40:39.242201858
16    230 days 12:24:12.441612772
17    101 days 02:50:23.713998446
21     63 days 09:10:07.319972534
22                            NaT
23     64 days 16:43:00.290223665
24     60 days 19:42:31.546736226
25    294 days 04:55:58.548387456
26    498 days 03:42:32.319606984
27   3669 days 21:24:59.730401664
28    323 days 20:56:49.935717436
29   1228 days 15:12:32.606227936
31    829 days 18:54:39.516933264
32    560 days 06:13:44.534938512
33    233 days 09:02:14.912258676
35   2205 days 05:01:30.803652128
41   1232 days 00:36:56.134497456
42     67 days 06:25:46.127511069
43     12 days 17:28:12.466101318
50    597 days 01:10:10.883395192
51   1085 days 02:36:30.959869728
52     70 days 23:04:40.279719361
53    454 days 11:56:59.198895008
Name: ATRASO_NOT, dtype: timedelta64[ns]
